In [1]:
import base64
import requests
from bs4 import BeautifulSoup as bs


class MakeHtml:
    def __init__(self):
        # imgBB api key
        imgApiKey = ""
        # user's confluence space baseurl
        self.baseUrl = "https://{userSpaceName}.atlassian.net/wiki"
        # header for api requests
        self.headers = {"Authorization": "Basic {token}"}
        # spaceListDict {"sample": {"id":1234, "key": "SPACE1", "selfUrl": "https://example.com"}}
        self.spaces ={}
        # contentListDict {"id(num)": "title"}
        self.contents = {}
        # htmlBodySoup
        self.htmlSoup = ""
    
    # send restapi requests to confluencer
    def sendRequest(self, url, method, paramDic={}, dataDic={}):
        if (method == "GET"):
            res = requests.get(url, params=paramDic, data=dataDic, headers=self.headers, allow_redirects=True)
        elif (method == "POST"):
            res = requests.post(url, params=paramDic, data=dataDic, headers=self.headers, allow_redirects=True)
        else :
            res = False
        return res
    
    # fill api access information and check api can speak
    def loginProcess(self, email, apiKey, userName, imgApiKey):
        # make token with user's email and api key
        secretText = "{}:{}".format(email, apiKey)
        token = base64.b64encode(secretText.encode('ascii')).decode('ascii')
        self.headers["Authorization"] = "Basic {}".format(token)
        self.imgApiKey = imgApiKey
        # make api url with fill username
        self.baseUrl = "https://{}.atlassian.net/wiki".format(userName)
        # check api
        testUrl = "{}/rest/api/space".format(self.baseUrl)
        res = self.sendRequest(testUrl, "GET")
        return res.status_code
    
    # get space list from 
    def getSpaceList(self):
        spaceListUrl = "{}/rest/api/space".format(self.baseUrl)
        res = self.sendRequest(spaceListUrl, "GET").json()
        for item in res['results']:
            self.spaces[item['name']] = {}
            self.spaces[item['name']]['id'] = item['id']
            self.spaces[item['name']]['key'] = item['key']
            self.spaces[item['name']]['self'] = item['_links']['self']
        return self.spaces

    def getContentList(self, spaceKey):
        params = {"spaceKey": spaceKey}
        contentListUrl = "{}/rest/api/content".format(self.baseUrl)
        res = self.sendRequest(contentListUrl, "GET", params).json()
        for item in res['results']:
            self.contents[item['id']] = item['title']
        return self.contents
    
    def getCententHtml(self, contentId):
        viewType = "view"
        params = {"expand": "body.{}".format(viewType)}
        contentUrl = "{}/rest/api/content/{}".format(self.baseUrl, contentId)
        res = self.sendRequest(contentUrl, "GET", params).json()
        htmlBody = res['body'][viewType]['value']
        self.htmlSoup = bs(htmlBody, 'html.parser')
        print (self.htmlSoup)
        
    def rebuildFormat(self):
        # replace all 'th' tag to 'tr' tag
        for th in self.htmlSoup.findAll('th'):
            th.name = 'td'
        
        # beautify table view
        for table in self.htmlSoup.findAll('table'):
            table.attrs['data-ke-style'] = 'style12'
           
        # append <code> tag inside <pre> tag for beautify code block
        for pre in self.htmlSoup.findAll('pre', class_='syntaxhighlighter-pre'):
            newTag = self.htmlSoup.new_tag('code')
            newTag.string = pre.string
            pre.string = ""
            pre.append(newTag)

        # use only body and add cover
#         customBody = self.htmlSoup.find('body')
#         customBody.attrs = {'id':'Content', 'style': 'padding: 5px;'}
#         customBody.name = 'div'
#         self.htmlSoup = customBody
            
    # upload image to imgbb and return image's url
    def uploadImg(self, imgBin):
        imgUpload = "https://api.imgbb.com/1/upload"
        param = {'key': self.imgApiKey}
        imgBase64 = base64.b64encode(imgBin).decode('ascii')
        data = {'image': imgBase64}
        res = self.sendRequest(imgUpload, "POST", param, data).json()
        return res['data']['url']
        
    # replace <img> tag for public viwer
    def rebuildImgStore(self):
        for img in self.htmlSoup.findAll('img'):
            imgSrc = img.attrs['src']
            res = self.sendRequest(imgSrc, "GET")
            imgUrl = self.uploadImg(res.content)
            # replace <img> tag's src and delete not use attrs
            img.attrs['src'] = imgUrl
            img.attrs['data-image-src'] = ""
            img.attrs['data-base-url'] = ""
    
    def saveHtmlFile(self, fileName):
        with open("{}.html".format(fileName), "w", -1, "utf-8") as f:
            f.write(str(self.htmlSoup))
        


In [7]:
from secrets import Secret

secret = Secret()
email = secret.email
apiKeyConfluence = secret.apiKeyConfluence
apiKeyImgbb = secret.apiKeyImgbb
userName = "geunsam2"

make1 = MakeHtml()
make1.loginProcess(email, apiKeyConfluence, userName, apiKeyImgbb)
make1.getSpaceList()
lists = make1.getContentList("GEUNSAM2")
print (lists)


{'33156': 'CNI와 Docker', '163844': 'Calico?Weave? CNI에 관하여', '229378': 'geunsam2', '884816': 'CNI 동작에 관하여', '1933313': 'Ingress와 LoadBalancer', '7897089': 'gitlab 컨테이너 backup/restore(백업 및 복원)', '8880168': '[confluence to tistory-1]API 조사', '14647297': 'DCOS Marathon-autoscaler and 및 or 옵션 사용', '14778444': '[confluence to tistory-2]css', '14778479': '[confluence to tistory-2]html 다듬기'}


In [24]:
make1.getCententHtml('8880168')
make1.rebuildFormat()
make1.rebuildImgStore()
make1.saveHtmlFile("SUCC")

<hr/><h2 id="confluencetotistory제작-1-개발1일차">개발 1일차</h2><h3 id="confluencetotistory제작-1-단위목표">단위목표</h3><ul><li><p>confluence api 조사</p></li><li><p>페이지, 스페이스 목록 정보 조회 api 확인</p></li><li><p>특정 페이지 <code>html</code> or <code>markdown</code> export api 확인 </p></li></ul><h3 id="confluencetotistory제작-1-RESTAPI활용">REST API 활용</h3><ul><li><p>인증 구현</p></li></ul><ol><li><p>personal token 생성</p></li><li><p><code>&lt;로그인 이메일&gt;:&lt;토큰&gt;</code> 을 base64로 인코딩</p></li><li><p>Authorization 헤더에 Basic &lt;base64 인코딩 된 값&gt;을 추가하여 사용</p></li></ol><div class="code panel pdl conf-macro output-block" data-hasbody="true" data-macro-id="52a38334-803d-4690-bc84-743de3ed8728" data-macro-name="code" style="border-width: 1px;"><div class="codeContent panelContent pdl">
<pre class="syntaxhighlighter-pre" data-syntaxhighlighter-params="brush: java; gutter: false; theme: Confluence" data-theme="Confluence">curl -H "Authorization: Basic &lt;yourToken&gt;" https://{confluenceBaseUrl}/rest/api/content</pre>
</div></d